In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Variant: Run with 1 million sample rows (second phase)

This variant includes `SAMPLE_ROWS=1000000`. To actually restrict the dataset to 100k rows, add `.sample(SAMPLE_ROWS, random_state=42)` to the cell that loads the CSV (or adapt that read to use SAMPLE_ROWS). The rest of the notebook is unchanged and contains detailed explanations before each original code cell.

In [2]:
SAMPLE_ROWS = 1000000  # set to 1000000 to use a 1 million-sample in the data-loading step


# Amazon Hybrid Recommender — Final Notebook

Comprehensive, runnable notebook containing prototype and scale stages, candidate generation, feature engineering, hard-negative mining, LightGBM ranking, diagnostics, and recommendations. This version contains full code and explanatory markdowns. Run cells sequentially. Some cells are resource-heavy (SBERT encoding, ALS, FAISS). The notebook includes CPU fallbacks for environments without `implicit` or `faiss`.

## How to run

1. Run top-to-bottom. If your environment is offline or lacks `implicit`/`faiss`, the notebook falls back to CPU-friendly alternatives.
2. The notebook writes artifact files to `OUT_DIR` (default `/kaggle/working`).
3. If you install new packages mid-run (e.g., `implicit`), you may need to restart the kernel.


### Explanation for Cell 3

**Purpose:** Imports libraries used later.

**Line-by-line (short):**
- `# Cell 1: Setup — Install-check and imports (non-blocking installs)` → Comment describing intent.
- `# Optional: uncomment pip installs if you have internet and want to ensure all libs are present.` → Comment describing intent.
- `!pip install -q datasets sentence-transformers scikit-learn lightgbm optuna joblib tqdm pandas scipy faiss-cpu implicit` → Performs a step relevant to data processing or modelling.
- `import os, time, warnings, importlib` → Imports module(s).
- `import numpy as np, pandas as pd, random, joblib` → Imports module(s).
- `from collections import Counter, defaultdict` → Imports module(s).
- `from scipy.sparse import csr_matrix` → Imports module(s).
- `from tqdm.auto import tqdm` → Imports module(s).
- `from math import log2` → Imports module(s).
- `warnings.filterwarnings('ignore', category=UserWarning)` → Performs a step relevant to data processing or modelling.
- `# Feature list and output directory (kept consistent with original notebook)` → Comment describing intent.
- `OUT_DIR = '/kaggle/working'` → Performs a step relevant to data processing or modelling.

...remaining lines omitted for brevity. See code cell for full content.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [3]:
# Cell 1: Setup — Install-check and imports (non-blocking installs)
# Optional: uncomment pip installs if you have internet and want to ensure all libs are present.
!pip install -q datasets sentence-transformers scikit-learn lightgbm optuna joblib tqdm pandas scipy faiss-cpu implicit

import os, time, warnings, importlib
import numpy as np, pandas as pd, random, joblib
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from tqdm.auto import tqdm
from math import log2

warnings.filterwarnings('ignore', category=UserWarning)

# Feature list and output directory (kept consistent with original notebook)
OUT_DIR = '/kaggle/working'
feature_names = ['sem_score','sem_rank','als_score','als_rank','svd_score','svd_rank','pop_log','pop_rank','recency_days','title_len','txt_len','user_mean_rating','user_activity_count']
feat_cols = feature_names
now_ts = int(time.time() * 1000)

# Flags for optional libs — later cells use these flags to decide runtime paths
HAS_IMPLICIT = importlib.util.find_spec('implicit') is not None
HAS_FAISS = importlib.util.find_spec('faiss') is not None
HAS_SBERT = importlib.util.find_spec('sentence_transformers') is not None

print('Environment flags — HAS_SBERT:', HAS_SBERT, 'HAS_IMPLICIT:', HAS_IMPLICIT, 'HAS_FAISS:', HAS_FAISS)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## 1 — Prototype Data Load (10k)

Load a small streaming sample (10k rows) from the Hugging Face Amazon Reviews dataset to iterate quickly and debug logic. This cell normalizes timestamps to milliseconds, drops records missing `asin` or `reviewerID`, and writes `sampled_reviews_10k.parquet` to `OUT_DIR`.

### Explanation for Cell 5

**Purpose:** Imports libraries used later. Saves results or models to disk.

**Line-by-line (short):**
- `# Prototype Data Load (10k)` → Comment describing intent.
- `SAMPLE_N = 10000` → Performs a step relevant to data processing or modelling.
- `HF_DATASET = 'McAuley-Lab/Amazon-Reviews-2023'` → Performs a step relevant to data processing or modelling.
- `HF_CONFIG = 'raw_review_Electronics'` → Performs a step relevant to data processing or modelling.
- `OUT_DIR = OUT_DIR if 'OUT_DIR' in globals() else '/kaggle/working'` → Performs a step relevant to data processing or modelling.
- `from datasets import load_dataset` → Imports module(s).
- `rows = []` → Performs a step relevant to data processing or modelling.
- `cnt = 0` → Performs a step relevant to data processing or modelling.
- `t0 = time.time()` → Performs a step relevant to data processing or modelling.
- `ds_iter = load_dataset(HF_DATASET, HF_CONFIG, split='full', streaming=True, trust_remote_code=True)` → Performs a step relevant to data processing or modelling.
- `for rec in ds_iter:` → Performs a step relevant to data processing or modelling.
- `try:` → Performs a step relevant to data processing or modelling.

...remaining lines omitted for brevity. See code cell for full content.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [4]:
# Prototype Data Load (10k)
SAMPLE_N = 10000
HF_DATASET = 'McAuley-Lab/Amazon-Reviews-2023'
HF_CONFIG = 'raw_review_Electronics'
OUT_DIR = OUT_DIR if 'OUT_DIR' in globals() else '/kaggle/working'

from datasets import load_dataset

rows = []
cnt = 0
t0 = time.time()
ds_iter = load_dataset(HF_DATASET, HF_CONFIG, split='full', streaming=True, trust_remote_code=True)
for rec in ds_iter:
    try:
        asin = rec.get('asin') or rec.get('parent_asin')
        user = rec.get('user_id') or rec.get('reviewerID')
        text = rec.get('text') or rec.get('reviewText') or ''
        overall = rec.get('rating') or rec.get('overall')
        ts = rec.get('timestamp') or rec.get('unixReviewTime') or 0
        helpful = rec.get('helpful_vote') or 0
        if asin is None or user is None: 
            continue
        ts_i = int(ts); ts_i = ts_i * 1000 if ts_i < 10**10 else ts_i
        rows.append({'asin': str(asin), 'reviewerID': str(user), 'reviewText': str(text), 'overall': float(overall) if overall is not None else float('nan'), 'unixReviewTime': int(ts_i), 'helpful_vote': int(helpful)})
        cnt += 1
        if cnt >= SAMPLE_N: break
    except Exception:
        continue
df = pd.DataFrame(rows)
os.makedirs(OUT_DIR, exist_ok=True)
df.to_parquet(f"{OUT_DIR}/sampled_reviews_10k.parquet", index=False)
print('Loaded', len(df), 'rows. Saved to', f"{OUT_DIR}/sampled_reviews_10k.parquet")

README.md: 0.00B [00:00, ?B/s]

Amazon-Reviews-2023.py: 0.00B [00:00, ?B/s]

Loaded 10000 rows. Saved to /kaggle/working/sampled_reviews_10k.parquet


### Build maps and interaction (prototype)

Create `item_map`, `user_map` and the sparse interaction matrix used by ALS and candidate generation.

### Explanation for Cell 7

**Purpose:** Performs an operation relevant to the pipeline (see code).

**Line-by-line (short):**
- `# Build maps and interaction (prototype)` → Comment describing intent.
- `unique_asins = sorted(df['asin'].unique().tolist())` → Performs a step relevant to data processing or modelling.
- `unique_users = sorted(df['reviewerID'].unique().tolist())` → Performs a step relevant to data processing or modelling.
- `item_map = {a:i for i,a in enumerate(unique_asins)}` → Performs a step relevant to data processing or modelling.
- `inv_item_map = {i:a for a,i in item_map.items()}` → Performs a step relevant to data processing or modelling.
- `user_map = {u:i for i,u in enumerate(unique_users)}` → Performs a step relevant to data processing or modelling.
- `inv_user_map = {i:u for u,i in user_map.items()}` → Performs a step relevant to data processing or modelling.
- `n_users, n_items = len(user_map), len(item_map)` → Performs a step relevant to data processing or modelling.
- `u_idx = []; i_idx = []; data = []` → Performs a step relevant to data processing or modelling.
- `for _, r in df.iterrows():` → Performs a step relevant to data processing or modelling.
- `try:` → Performs a step relevant to data processing or modelling.
- `u = user_map[r['reviewerID']]; i = item_map[r['asin']]` → Performs a step relevant to data processing or modelling.

...remaining lines omitted for brevity. See code cell for full content.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [5]:
# Build maps and interaction (prototype)
unique_asins = sorted(df['asin'].unique().tolist())
unique_users = sorted(df['reviewerID'].unique().tolist())
item_map = {a:i for i,a in enumerate(unique_asins)}
inv_item_map = {i:a for a,i in item_map.items()}
user_map = {u:i for i,u in enumerate(unique_users)}
inv_user_map = {i:u for u,i in user_map.items()}

n_users, n_items = len(user_map), len(item_map)
u_idx = []; i_idx = []; data = []
for _, r in df.iterrows():
    try:
        u = user_map[r['reviewerID']]; i = item_map[r['asin']]
        w = float(r['overall']) if not pd.isna(r['overall']) else 1.0 + int(r.get('helpful_vote',0))
        u_idx.append(u); i_idx.append(i); data.append(w)
    except KeyError:
        continue

interaction = csr_matrix((data, (u_idx, i_idx)), shape=(n_users, n_items)).tocsr()
item_user_matrix = interaction.T.tocsr()
pop_counts = Counter(df['asin'])
test_df_proto = df.groupby('reviewerID').tail(1).reset_index(drop=True)
print('Maps built:', n_users, 'users x', n_items, 'items')

Maps built: 1561 users x 8907 items


## 2 — Helper functions & Feature Builder

This cell defines helper utilities used throughout the notebook:

- `user_profile_emb`: compute a user's profile embedding (mean of item embeddings) with safe fallbacks.
- `build_candidates_union`: aggregate candidates from semantic nearest neighbors, ALS, SVD, popularity and MF neighbors.
- `build_features_for_candidates`: compute features used by the LightGBM reranker.

All functions include robust error handling so the notebook runs even if optional libraries are missing.

### Explanation for Cell 9

**Purpose:** Imports libraries used later. Defines function(s) used downstream.

**Line-by-line (short):**
- `# Helper functions and feature builder (robust)` → Comment describing intent.
- `import numpy as np, pandas as pd, math` → Imports module(s).
- `from collections import defaultdict` → Imports module(s).
- `# Ensure inv maps exist` → Comment describing intent.
- `if 'inv_item_map' not in globals(): inv_item_map = {}` → Performs a step relevant to data processing or modelling.
- `if 'inv_user_map' not in globals(): inv_user_map = {}` → Performs a step relevant to data processing or modelling.
- `def user_profile_emb(uidx, emb_array, df_data, i_map, u_map):` → Performs a step relevant to data processing or modelling.
- `# uid -> user string` → Comment describing intent.
- `uid = None` → Performs a step relevant to data processing or modelling.
- `if 'inv_user_map' in globals() and uidx in inv_user_map:` → Performs a step relevant to data processing or modelling.
- `uid = inv_user_map.get(uidx)` → Performs a step relevant to data processing or modelling.
- `if uid is None:` → Performs a step relevant to data processing or modelling.

...remaining lines omitted for brevity. See code cell for full content.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [6]:
# Helper functions and feature builder (robust)
import numpy as np, pandas as pd, math
from collections import defaultdict

# Ensure inv maps exist
if 'inv_item_map' not in globals(): inv_item_map = {}
if 'inv_user_map' not in globals(): inv_user_map = {}

def user_profile_emb(uidx, emb_array, df_data, i_map, u_map):
    # uid -> user string
    uid = None
    if 'inv_user_map' in globals() and uidx in inv_user_map:
        uid = inv_user_map.get(uidx)
    if uid is None:
        # fallback: if uidx already a string, try that
        uid = uidx if isinstance(uidx, str) else None
    if uid is None:
        return emb_array.mean(axis=0)
    item_list = df_data[df_data['reviewerID']==uid]['asin'].map(i_map).dropna().astype(int).tolist()
    if item_list and len(item_list)>0:
        return emb_array[item_list].mean(axis=0)
    else:
        return emb_array.mean(axis=0)

def build_candidates_union(uidx, emb_array, als_model, svd_facs, i_map, p_counts, u_map, df_data, inter_mat, top_sem=500, top_als=200, top_svd=200, top_pop=200, top_mf=200, max_cands=2000):
    candidates = []; seen = set()
    # semantic
    if emb_array is not None:
        try:
            up = user_profile_emb(uidx, emb_array, df_data, i_map, u_map).astype('float32')
            un = up / (np.linalg.norm(up) + 1e-9)
            scores = emb_array.dot(un)
            sem_idxs = np.argsort(-scores)[:top_sem]
            for it in map(int, sem_idxs):
                if it not in seen: seen.add(it); candidates.append(it)
        except Exception:
            pass
    # ALS recommend
    if als_model is not None:
        try:
            rec = als_model.recommend(uidx, inter_mat[uidx], N=top_als)
            als_list = []
            if isinstance(rec, list) and len(rec)>0 and isinstance(rec[0], (list,tuple)):
                als_list = [int(r[0]) for r in rec]
            elif isinstance(rec, tuple) and len(rec)==2:
                als_list = [int(x) for x in rec[0]]
            else:
                try:
                    als_list = [int(r[0]) if isinstance(r,(list,tuple)) else int(r) for r in rec]
                except Exception:
                    als_list = []
            for it in als_list:
                if it not in seen: seen.add(it); candidates.append(it)
        except Exception:
            pass
    # SVD global similarity
    if svd_facs is not None:
        try:
            user_vec = svd_facs.mean(axis=0)
            svd_scores = svd_facs.dot(user_vec)
            svd_idx = np.argsort(-svd_scores)[:top_svd]
            for it in map(int, svd_idx):
                if it not in seen: seen.add(it); candidates.append(it)
        except Exception:
            pass
    # popularity
    try:
        pop_list = [i_map.get(a) for a,_ in p_counts.most_common(top_pop) if a in i_map]
        for it in pop_list:
            if it not in seen: seen.add(it); candidates.append(it)
    except Exception:
        pass
    # MF neighbors (approx via item factors if available)
    try:
        if als_model is not None and hasattr(als_model, 'item_factors'):
            item_facs = als_model.item_factors
            user_items = inter_mat[uidx].indices.tolist() if hasattr(inter_mat[uidx], 'indices') else []
            for it in user_items[:5]:
                vec = item_facs[it]
                sim = item_facs.dot(vec)
                neigh = np.argsort(-sim)[:top_mf]
                for ni in map(int, neigh):
                    if ni not in seen: seen.add(ni); candidates.append(ni)
    except Exception:
        pass
    return candidates[:max_cands]

def build_features_for_candidates(uidx, cands, emb_array, als_facs, als_u_facs, svd_facs, df_data, i_map, u_map, p_counts, feat_names, now_ts):
    n = len(cands); X = np.zeros((n, len(feat_names)), dtype=float)
    # user emb
    try:
        uemb = user_profile_emb(uidx, emb_array, df_data, i_map, u_map); uembn = uemb/(np.linalg.norm(uemb)+1e-9)
    except Exception:
        uembn = emb_array.mean(axis=0); uembn = uembn/(np.linalg.norm(uembn)+1e-9)
    # sem
    try:
        sem_scores = (emb_array[cands] * uembn).sum(axis=1)
        X[:, feat_names.index('sem_score')] = sem_scores
        ranks = np.empty(n, dtype=int); ranks[np.argsort(-sem_scores)] = np.arange(n); X[:, feat_names.index('sem_rank')] = ranks
    except Exception:
        pass
    # als
    try:
        if als_facs is not None and als_u_facs is not None:
            uf = als_u_facs[uidx] if uidx < als_u_facs.shape[0] else als_facs.mean(axis=0)
            als_scores = np.dot(als_facs[cands], uf)
            X[:, feat_names.index('als_score')] = als_scores
            ranks = np.empty(n, dtype=int); ranks[np.argsort(-als_scores)] = np.arange(n); X[:, feat_names.index('als_rank')] = ranks
    except Exception:
        pass
    # svd
    try:
        if svd_facs is not None:
            user_fact = svd_facs.mean(axis=0); svd_scores = svd_facs[cands].dot(user_fact)
            X[:, feat_names.index('svd_score')] = svd_scores; ranks = np.empty(n, dtype=int); ranks[np.argsort(-svd_scores)] = np.arange(n); X[:, feat_names.index('svd_rank')] = ranks
    except Exception:
        pass
    # pop & recency
    try:
        pop_vals = np.array([p_counts.get(inv_item_map.get(i, ''), 0) for i in cands], dtype=float)
        X[:, feat_names.index('pop_log')] = np.log1p(pop_vals); ranks = np.empty(n,dtype=int); ranks[np.argsort(-pop_vals)] = np.arange(n); X[:, feat_names.index('pop_rank')] = ranks
        recency_ms = np.array([item_last_ts.get(inv_item_map.get(i, ''), 0) for i in cands], dtype=float); X[:, feat_names.index('recency_days')] = ((now_ts - recency_ms)/(1000*60*60*24)).clip(min=0)
    except Exception:
        pass
    # text & user stats
    try:
        title_len=[]; txt_len=[]
        for idx in cands:
            asin = inv_item_map.get(idx, ''); rows_sub = df_data[df_data['asin']==asin]
            title = rows_sub['title'].iloc[0] if ('title' in rows_sub.columns and len(rows_sub)>0 and pd.notna(rows_sub['title'].iloc[0])) else ''
            text = rows_sub['reviewText'].iloc[0] if ('reviewText' in rows_sub.columns and len(rows_sub)>0 and pd.notna(rows_sub['reviewText'].iloc[0])) else ''
            title_len.append(len(str(title))); txt_len.append(len(str(text)))
        X[:, feat_names.index('title_len')] = np.array(title_len); X[:, feat_names.index('txt_len')] = np.array(txt_len)
        uid = inv_user_map.get(uidx, None); user_rows = df_data[df_data['reviewerID']==uid] if uid else pd.DataFrame()
        X[:, feat_names.index('user_mean_rating')] = float(user_rows['overall'].mean() if len(user_rows)>0 else 0.0); X[:, feat_names.index('user_activity_count')] = float(len(user_rows))
    except Exception:
        pass
    return {'X': X, 'cands': cands, 'feature_names': feat_names}

## 3 — Hard-negative mining (prototype)

Create meta-training pairs (pos/neg) for the LTR model. This prototype uses a combination of hard negatives (top-ranked non-pos by ALS/SVD) and random negatives.

### Explanation for Cell 11

**Purpose:** Imports libraries used later.

**Line-by-line (short):**
- `# Hard-negative mining (prototype)` → Comment describing intent.
- `import random` → Imports module(s).
- `USERS_TO_USE = 300; HARD_NEG_PER_POS = 60; RANDOM_NEG_PER_POS = 5` → Performs a step relevant to data processing or modelling.
- `meta_rows = []; all_users = list(df['reviewerID'].unique()); sample_users = all_users[:min(USERS_TO_USE, len(all_users))]` → Performs a step relevant to data processing or modelling.
- `for uid in tqdm(sample_users, desc='Building Meta Prototype'):` → Performs a step relevant to data processing or modelling.
- `uidx = user_map.get(uid);` → Performs a step relevant to data processing or modelling.
- `if uidx is None: continue` → Performs a step relevant to data processing or modelling.
- `cands = build_candidates_union(uidx, None, None, None, item_map, pop_counts, user_map, df, interaction, top_sem=500, top_als=200, top_svd=200, top_pop=200, max_cands=1000)` → Performs a step relevant to data processing or modelling.
- `if not cands: continue` → Performs a step relevant to data processing or modelling.
- `mf = build_features_for_candidates(uidx, cands, np.zeros((len(item_map), 768)), None, None, None, df, item_map, user_map, pop_counts, feature_names, now_ts)` → Performs a step relevant to data processing or modelling.
- `Xc = mf['X']; cidx = mf['cands']; fns = mf['feature_names']` → Performs a step relevant to data processing or modelling.
- `true_asins = test_df_proto[test_df_proto['reviewerID']==uid]['asin'].tolist(); true_idxs = [item_map[a] for a in true_asins if a in item_map]` → Performs a step relevant to data processing or modelling.

...remaining lines omitted for brevity. See code cell for full content.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [7]:
# Hard-negative mining (prototype)
import random
USERS_TO_USE = 300; HARD_NEG_PER_POS = 60; RANDOM_NEG_PER_POS = 5
meta_rows = []; all_users = list(df['reviewerID'].unique()); sample_users = all_users[:min(USERS_TO_USE, len(all_users))]
for uid in tqdm(sample_users, desc='Building Meta Prototype'):
    uidx = user_map.get(uid); 
    if uidx is None: continue
    cands = build_candidates_union(uidx, None, None, None, item_map, pop_counts, user_map, df, interaction, top_sem=500, top_als=200, top_svd=200, top_pop=200, max_cands=1000)
    if not cands: continue
    mf = build_features_for_candidates(uidx, cands, np.zeros((len(item_map), 768)), None, None, None, df, item_map, user_map, pop_counts, feature_names, now_ts)
    Xc = mf['X']; cidx = mf['cands']; fns = mf['feature_names']
    true_asins = test_df_proto[test_df_proto['reviewerID']==uid]['asin'].tolist(); true_idxs = [item_map[a] for a in true_asins if a in item_map]
    if not true_idxs: continue
    als_col = Xc[:, fns.index('als_score')] if 'als_score' in fns else np.zeros(Xc.shape[0])
    svd_col = Xc[:, fns.index('svd_score')] if 'svd_score' in fns else np.zeros(Xc.shape[0])
    hard_score = 0.5 * als_col + 0.5 * svd_col
    cand_arr = np.array(cidx); pos_mask = np.isin(cand_arr, true_idxs); non_pos_idx = np.where(~pos_mask)[0]
    if len(non_pos_idx)==0: continue
    ordering = non_pos_idx[np.argsort(-hard_score[non_pos_idx])]; hard_negs_idx = ordering[:HARD_NEG_PER_POS].tolist()
    remaining = list(set(non_pos_idx.tolist()) - set(hard_negs_idx)); random_negs_idx = random.sample(remaining, min(RANDOM_NEG_PER_POS, len(remaining))) if remaining else []
    for t in true_idxs:
        pos_loc = np.where(cand_arr==t)[0]; 
        if len(pos_loc)==0: continue
        for p in pos_loc:
            row = {'user_idx': uidx, 'item_idx': int(cidx[p]), 'label': 1}
            for j,fn in enumerate(fns):
                try: row[fn] = float(Xc[p,j])
                except: row[fn] = 0.0
            meta_rows.append(row)
            for idx in hard_negs_idx:
                rn = {'user_idx': uidx, 'item_idx': int(cidx[idx]), 'label': 0}
                for j,fn in enumerate(fns):
                    try: rn[fn] = float(Xc[idx,j])
                    except: rn[fn] = 0.0
                meta_rows.append(rn)
            for idx in random_negs_idx:
                rr = {'user_idx': uidx, 'item_idx': int(cidx[idx]), 'label': 0}
                for j,fn in enumerate(fns):
                    try: rr[fn] = float(Xc[idx,j])
                    except: rr[fn] = 0.0
                meta_rows.append(rr)
meta_df_small = pd.DataFrame(meta_rows).sample(frac=1.0, random_state=42).reset_index(drop=True)
meta_df_small.to_parquet(f"{OUT_DIR}/meta_enhanced_small.parquet", index=False)
print('meta_df_small shape:', meta_df_small.shape)

Building Meta Prototype:   0%|          | 0/300 [00:00<?, ?it/s]

meta_df_small shape: (2640, 16)


## 4 — LightGBM Tuning & Prototype Training

Use Optuna to tune lambdarank hyperparameters. Important: set `feature_pre_filter=False` so trials changing `min_data_in_leaf` don't error.

### Explanation for Cell 13

**Purpose:** Imports libraries used later. Defines function(s) used downstream. Trains a model / fits parameters. Produces predictions on data.

**Line-by-line (short):**
- `# LightGBM tuning & training (prototype)` → Comment describing intent.
- `import lightgbm as lgb, optuna` → Imports module(s).
- `from sklearn.model_selection import train_test_split` → Imports module(s).
- `from math import log2` → Imports module(s).
- `feat_cols = [c for c in meta_df_small.columns if c not in ('user_idx','item_idx','label')]` → Performs a step relevant to data processing or modelling.
- `users = meta_df_small['user_idx'].unique(); train_u, valid_u = train_test_split(list(users), test_size=0.20, random_state=42)` → Performs a step relevant to data processing or modelling.
- `tr_mask = meta_df_small['user_idx'].isin(train_u); val_mask = meta_df_small['user_idx'].isin(valid_u)` → Performs a step relevant to data processing or modelling.
- `X_tr = meta_df_small[tr_mask][feat_cols].values; y_tr = meta_df_small[tr_mask]['label'].values` → Performs a step relevant to data processing or modelling.
- `gr_tr = meta_df_small[tr_mask].groupby('user_idx', sort=False).size().astype(int).values` → Groups and aggregates data.
- `X_val = meta_df_small[val_mask][feat_cols].values; y_val = meta_df_small[val_mask]['label'].values` → Performs a step relevant to data processing or modelling.
- `gr_val = meta_df_small[val_mask].groupby('user_idx', sort=False).size().astype(int).values` → Groups and aggregates data.
- `dtrain = lgb.Dataset(X_tr, label=y_tr, group=gr_tr); dval = lgb.Dataset(X_val, label=y_val, group=gr_val, reference=dtrain)` → Performs a step relevant to data processing or modelling.

...remaining lines omitted for brevity. See code cell for full content.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [8]:
# LightGBM tuning & training (prototype)
import lightgbm as lgb, optuna
from sklearn.model_selection import train_test_split
from math import log2

feat_cols = [c for c in meta_df_small.columns if c not in ('user_idx','item_idx','label')]
users = meta_df_small['user_idx'].unique(); train_u, valid_u = train_test_split(list(users), test_size=0.20, random_state=42)
tr_mask = meta_df_small['user_idx'].isin(train_u); val_mask = meta_df_small['user_idx'].isin(valid_u)
X_tr = meta_df_small[tr_mask][feat_cols].values; y_tr = meta_df_small[tr_mask]['label'].values
gr_tr = meta_df_small[tr_mask].groupby('user_idx', sort=False).size().astype(int).values
X_val = meta_df_small[val_mask][feat_cols].values; y_val = meta_df_small[val_mask]['label'].values
gr_val = meta_df_small[val_mask].groupby('user_idx', sort=False).size().astype(int).values
dtrain = lgb.Dataset(X_tr, label=y_tr, group=gr_tr); dval = lgb.Dataset(X_val, label=y_val, group=gr_val, reference=dtrain)

def ndcg_per_group(preds, df_masked, K=10):
    eval_users = df_masked['user_idx'].unique(); idx = 0; ndcgs = []
    for u in eval_users:
        sub = df_masked[df_masked['user_idx']==u]; n = len(sub)
        if n==0: continue
        p = preds[idx: idx + n]; items = sub['item_idx'].values; order = np.argsort(-p); ranked = items[order].tolist()
        true_items = sub['item_idx'][sub['label']==1].tolist()
        gains = [1.0 if it in true_items else 0.0 for it in ranked[:K]]
        dcg = sum(g / log2(i+2) for i,g in enumerate(gains)); idcg = sum(1.0 / log2(i+2) for i in range(min(len(true_items), K))) if len(true_items)>0 else 0.0
        ndcgs.append(dcg/idcg if idcg>0 else 0.0); idx += n
    return float(np.mean(ndcgs)) if len(ndcgs)>0 else 0.0

def objective(trial):
    param = {'objective':'lambdarank','metric':'ndcg','ndcg_eval_at':[10],
             'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
             'num_leaves': trial.suggest_int('num_leaves', 31, 127),
             'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 200),
             'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
             'verbosity': -1, 'seed': 42, 'feature_pre_filter': False}
    try:
        bst = lgb.train(param, dtrain, num_boost_round=500, valid_sets=[dval], callbacks=[lgb.early_stopping(stopping_rounds=40), lgb.log_evaluation(period=0)])
        preds_val = bst.predict(X_val); df_val = meta_df_small[val_mask].copy().reset_index(drop=True)
        return -ndcg_per_group(preds_val, df_val, K=10)
    except Exception as e:
        print('LightGBM failed in objective:', e); return 1.0

study = optuna.create_study(direction='minimize'); study.optimize(objective, n_trials=20, show_progress_bar=True)
best_params_proto = study.best_params; print('Best params:', best_params_proto)

params = {'objective':'lambdarank','metric':'ndcg','ndcg_eval_at':[10,50],
          'learning_rate':float(best_params_proto.get('learning_rate',0.05)),
          'num_leaves':int(best_params_proto.get('num_leaves',63)),
          'min_data_in_leaf':int(best_params_proto.get('min_data_in_leaf',20)),
          'feature_fraction':float(best_params_proto.get('feature_fraction',0.8)),'verbosity':-1,'seed':42,'feature_pre_filter':False}
bst_proto = lgb.train(params, dtrain, num_boost_round=1000, valid_sets=[dval], callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=100)])
joblib.dump(bst_proto, f"{OUT_DIR}/lgbm_reranker_prototype.joblib")

[I 2025-10-02 08:21:25,060] A new study created in memory with name: no-name-a195d148-4b93-44c2-ac86-f6204bd4dbc1


  0%|          | 0/20 [00:00<?, ?it/s]

Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[1]	valid_0's ndcg@10: 0.669183
[I 2025-10-02 08:21:25,146] Trial 0 finished with value: -0.0 and parameters: {'learning_rate': 0.0758736964506041, 'num_leaves': 122, 'min_data_in_leaf': 176, 'feature_fraction': 0.6248639773775441}. Best is trial 0 with value: -0.0.
Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[25]	valid_0's ndcg@10: 0.714801
[I 2025-10-02 08:21:25,208] Trial 1 finished with value: -0.0 and parameters: {'learning_rate': 0.04107660439912675, 'num_leaves': 54, 'min_data_in_leaf': 80, 'feature_fraction': 0.7131998566962514}. Best is trial 0 with value: -0.0.
Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[9]	valid_0's ndcg@10: 0.790458
[I 2025-10-02 08:21:25,261] Trial 2 finished with value: -0.0 and parameters: {'learning_rate': 0.011692441653214036, 'num_leaves': 58, 'min_data_

['/kaggle/working/lgbm_reranker_prototype.joblib']

## 5 — Scale-up Data Load (100k) & Model Initialization

This cell builds a larger sample (100k rows) and computes final embeddings and factors. It prefers SBERT/implicit/FAISS when available but includes CPU-friendly fallbacks (TF-IDF + SVD and TruncatedSVD-based MF) so the notebook runs offline.

### Explanation for Cell 15

**Purpose:** Imports libraries used later. Trains a model / fits parameters.

**Line-by-line (short):**
- `# Scale-up data load (100k) with fallbacks` → Comment describing intent.
- `SCALE_N = 10000` → Performs a step relevant to data processing or modelling.
- `SBERT_MODEL_FINAL = 'all-mpnet-base-v2'` → Performs a step relevant to data processing or modelling.
- `from datasets import load_dataset` → Imports module(s).
- `ds_iter = load_dataset('McAuley-Lab/Amazon-Reviews-2023', 'raw_review_Electronics', split='full', streaming=True, trust_remote_code=True)` → Performs a step relevant to data processing or modelling.
- `rows = []; cnt = 0` → Performs a step relevant to data processing or modelling.
- `for rec in ds_iter:` → Performs a step relevant to data processing or modelling.
- `try:` → Performs a step relevant to data processing or modelling.
- `asin = rec.get('asin') or rec.get('parent_asin'); user = rec.get('user_id') or rec.get('reviewerID'); text = rec.get('text') or rec.get('reviewText') or ''` → Performs a step relevant to data processing or modelling.
- `overall = rec.get('rating') or rec.get('overall'); ts = rec.get('timestamp') or rec.get('unixReviewTime') or 0` → Performs a step relevant to data processing or modelling.
- `if asin is None or user is None: continue` → Performs a step relevant to data processing or modelling.
- `ts_i = int(ts); ts_i = ts_i*1000 if ts_i < 10**10 else ts_i` → Performs a step relevant to data processing or modelling.

...remaining lines omitted for brevity. See code cell for full content.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [9]:
# Scale-up data load (100k) with fallbacks
SCALE_N = 10000
SBERT_MODEL_FINAL = 'all-mpnet-base-v2'

from datasets import load_dataset
ds_iter = load_dataset('McAuley-Lab/Amazon-Reviews-2023', 'raw_review_Electronics', split='full', streaming=True, trust_remote_code=True)
rows = []; cnt = 0
for rec in ds_iter:
    try:
        asin = rec.get('asin') or rec.get('parent_asin'); user = rec.get('user_id') or rec.get('reviewerID'); text = rec.get('text') or rec.get('reviewText') or ''
        overall = rec.get('rating') or rec.get('overall'); ts = rec.get('timestamp') or rec.get('unixReviewTime') or 0
        if asin is None or user is None: continue
        ts_i = int(ts); ts_i = ts_i*1000 if ts_i < 10**10 else ts_i
        rows.append({'asin':str(asin),'reviewerID':str(user),'reviewText':str(text),'overall':float(overall) if overall is not None else float('nan'),'unixReviewTime':int(ts_i)})
        cnt += 1
        if cnt >= SCALE_N: break
    except Exception:
        continue
df_scale = pd.DataFrame(rows)
unique_asins_scale = sorted(df_scale['asin'].unique().tolist()); unique_users_scale = sorted(df_scale['reviewerID'].unique().tolist())
item_map_scale = {a:i for i,a in enumerate(unique_asins_scale)}; inv_item_map_scale = {i:a for a,i in item_map_scale.items()}
user_map_scale = {u:i for i,u in enumerate(unique_users_scale)}; inv_user_map_scale = {i:u for u,i in user_map_scale.items()}
n_users_scale, n_items_scale = len(user_map_scale), len(item_map_scale)
u_idx=[]; i_idx=[]; data=[]
for _, r in df_scale.iterrows():
    try:
        u = user_map_scale[r['reviewerID']]; i = item_map_scale[r['asin']]
        w = float(r['overall']) if not pd.isna(r['overall']) else 1.0; u_idx.append(u); i_idx.append(i); data.append(w)
    except KeyError: continue
interaction_scale = csr_matrix((data,(u_idx,i_idx)), shape=(n_users_scale,n_items_scale)).tocsr()
item_user_matrix_scale = interaction_scale.T.tocsr()
pop_counts_scale = Counter(df_scale['asin'])
test_df_scale = df_scale.groupby('reviewerID').tail(1).reset_index(drop=True)

# Build semantic embeddings: SBERT if available else TF-IDF+SVD fallback
if HAS_SBERT:
    from sentence_transformers import SentenceTransformer
    sbert_final = SentenceTransformer(SBERT_MODEL_FINAL, device='cpu')
    agg = defaultdict(list)
    for _, r in df_scale[['asin','reviewText']].iterrows():
        a = r['asin']
        if len(agg[a]) < 5: agg[a].append(str(r['reviewText'])[:1200])
    texts = [' . '.join(agg.get(a, [''])) for a in sorted(item_map_scale.keys(), key=lambda x: item_map_scale[x])]
    emb_batches = []; batch_size = 256
    for i in tqdm(range(0, len(texts), batch_size), desc='SBERT encode'):
        emb = sbert_final.encode(texts[i:i+batch_size], convert_to_numpy=True, show_progress_bar=False)
        emb_batches.append(emb)
    item_emb_final = np.vstack(emb_batches).astype('float32')
else:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.decomposition import TruncatedSVD as _TSVD
    agg = defaultdict(list)
    for _, r in df_scale[['asin','reviewText']].iterrows():
        a = r['asin']
        if len(agg[a]) < 5: agg[a].append(str(r['reviewText'])[:1200])
    texts = [' . '.join(agg.get(a, [''])) for a in sorted(item_map_scale.keys(), key=lambda x: item_map_scale[x])]
    tf = TfidfVectorizer(max_features=65536, ngram_range=(1,2), min_df=2)
    Xtf = tf.fit_transform(texts)
    svd_text = _TSVD(n_components=384, random_state=42)
    item_emb_final = svd_text.fit_transform(Xtf).astype('float32')

# Build MF factors: implicit ALS if available else TruncatedSVD fallback
if HAS_IMPLICIT:
    try:
        from implicit.als import AlternatingLeastSquares
        als_model_final = AlternatingLeastSquares(factors=128, regularization=0.1, iterations=20, use_gpu=False)
        als_model_final.fit(item_user_matrix_scale.astype('float32'))
        als_item_factors_final = als_model_final.item_factors; als_user_factors_final = als_model_final.user_factors
    except Exception:
        # fallback to SVD factors below
        HAS_IMPLICIT = False

if not HAS_IMPLICIT:
    from sklearn.decomposition import TruncatedSVD as _TSVD2
    print('implicit not available or failed; using TruncatedSVD fallback for MF factors')
    svd_mf = _TSVD2(n_components=128, random_state=42)
    als_item_factors_final = svd_mf.fit_transform(item_user_matrix_scale)
    # user factors: approximate by dividing R^T * F by counts
    try:
        R = item_user_matrix_scale
        user_num = (R.T).dot(als_item_factors_final)
        user_counts = np.array((R != 0).sum(axis=0)).reshape(-1)
        user_counts = np.where(user_counts == 0, 1.0, user_counts)
        als_user_factors_final = (user_num / user_counts[:, None]).astype('float32')
    except Exception:
        als_user_factors_final = np.random.normal(scale=0.01, size=(n_users_scale, als_item_factors_final.shape[1])).astype('float32')

svd_item_factors_final = als_item_factors_final  # reuse
item_emb_norm_final = item_emb_final / (np.linalg.norm(item_emb_final, axis=1, keepdims=True) + 1e-9)
print('Final item_emb shape:', item_emb_final.shape)

2025-10-02 08:21:39.649865: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759393299.863532      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759393299.926999      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SBERT encode:   0%|          | 0/35 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

Final item_emb shape: (8907, 768)


## 6 — Scale Hard-negative Mining & Final LTR Training

This cell mines hard negatives across many users and trains the final LightGBM reranker on the large meta dataset.

### Explanation for Cell 17

**Purpose:** Imports libraries used later. Trains a model / fits parameters.

**Line-by-line (short):**
- `# Scale hard-neg mining + LTR training` → Comment describing intent.
- `USERS_TO_USE_SCALE = 5000; HARD_NEG_PER_POS = 80; RANDOM_NEG_PER_POS = 5` → Performs a step relevant to data processing or modelling.
- `meta_rows_scale = []` → Performs a step relevant to data processing or modelling.
- `sample_users_scale = df_scale['reviewerID'].unique()[:min(USERS_TO_USE_SCALE, n_users_scale)]` → Performs a step relevant to data processing or modelling.
- `for uid in tqdm(sample_users_scale, desc='Building Meta Scale'):` → Performs a step relevant to data processing or modelling.
- `uidx = user_map_scale.get(uid);` → Performs a step relevant to data processing or modelling.
- `if uidx is None: continue` → Performs a step relevant to data processing or modelling.
- `cands = build_candidates_union(uidx, item_emb_final, (als_model_final if 'als_model_final' in globals() else None), svd_item_factors_final, item_map_scale, pop_counts_scale, user_map_scale, df_scale, interaction_scale, top_sem=1000, top_als=500, top_svd=500, top_pop=500, top_mf=200, max_cands=3000)` → Performs a step relevant to data processing or modelling.
- `if not cands: continue` → Performs a step relevant to data processing or modelling.
- `mf = build_features_for_candidates(uidx, cands, item_emb_final, (als_item_factors_final if 'als_item_factors_final' in globals() else None), (als_user_factors_final if 'als_user_factors_final' in globals() else None), svd_item_factors_final, df_scale, item_map_scale, user_map_scale, pop_counts_scale, feature_names, now_ts)` → Performs a step relevant to data processing or modelling.
- `Xc = mf['X']; cidx = mf['cands']; fns = mf['feature_names']` → Performs a step relevant to data processing or modelling.
- `true_asins = test_df_scale[test_df_scale['reviewerID']==uid]['asin'].tolist()` → Performs a step relevant to data processing or modelling.

...remaining lines omitted for brevity. See code cell for full content.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [10]:
# Scale hard-neg mining + LTR training
USERS_TO_USE_SCALE = 5000; HARD_NEG_PER_POS = 80; RANDOM_NEG_PER_POS = 5
meta_rows_scale = []
sample_users_scale = df_scale['reviewerID'].unique()[:min(USERS_TO_USE_SCALE, n_users_scale)]
for uid in tqdm(sample_users_scale, desc='Building Meta Scale'):
    uidx = user_map_scale.get(uid); 
    if uidx is None: continue
    cands = build_candidates_union(uidx, item_emb_final, (als_model_final if 'als_model_final' in globals() else None), svd_item_factors_final, item_map_scale, pop_counts_scale, user_map_scale, df_scale, interaction_scale, top_sem=1000, top_als=500, top_svd=500, top_pop=500, top_mf=200, max_cands=3000)
    if not cands: continue
    mf = build_features_for_candidates(uidx, cands, item_emb_final, (als_item_factors_final if 'als_item_factors_final' in globals() else None), (als_user_factors_final if 'als_user_factors_final' in globals() else None), svd_item_factors_final, df_scale, item_map_scale, user_map_scale, pop_counts_scale, feature_names, now_ts)
    Xc = mf['X']; cidx = mf['cands']; fns = mf['feature_names']
    true_asins = test_df_scale[test_df_scale['reviewerID']==uid]['asin'].tolist()
    true_idxs = [item_map_scale[a] for a in true_asins if a in item_map_scale]
    if not true_idxs: continue
    als_col = Xc[:, fns.index('als_score')] if 'als_score' in fns else np.zeros(Xc.shape[0])
    svd_col = Xc[:, fns.index('svd_score')] if 'svd_score' in fns else np.zeros(Xc.shape[0])
    hard_score = 0.5 * als_col + 0.5 * svd_col
    cand_arr = np.array(cidx); pos_mask = np.isin(cand_arr, true_idxs); non_pos_idx = np.where(~pos_mask)[0]
    if len(non_pos_idx)==0: continue
    ordering = non_pos_idx[np.argsort(-hard_score[non_pos_idx])]; hard_negs_idx = ordering[:HARD_NEG_PER_POS].tolist()
    remaining = list(set(non_pos_idx.tolist()) - set(hard_negs_idx)); random_negs_idx = random.sample(remaining, min(RANDOM_NEG_PER_POS, len(remaining))) if remaining else []
    for t in true_idxs:
        pos_loc = np.where(cand_arr==t)[0]; 
        if len(pos_loc)==0: continue
        for p in pos_loc:
            row = {'user_idx': uidx, 'item_idx': int(cidx[p]), 'label': 1}
            for j,fn in enumerate(fns):
                try: row[fn] = float(Xc[p,j])
                except: row[fn] = 0.0
            meta_rows_scale.append(row)
            for idx in hard_negs_idx:
                rn = {'user_idx': uidx, 'item_idx': int(cidx[idx]), 'label': 0}
                for j,fn in enumerate(fns):
                    try: rn[fn] = float(Xc[idx,j])
                    except: rn[fn] = 0.0
                meta_rows_scale.append(rn)
            for idx in random_negs_idx:
                rr = {'user_idx': uidx, 'item_idx': int(cidx[idx]), 'label': 0}
                for j,fn in enumerate(fns):
                    try: rr[fn] = float(Xc[idx,j])
                    except: rr[fn] = 0.0
                meta_rows_scale.append(rr)
meta_df_scale = pd.DataFrame(meta_rows_scale).sample(frac=1.0, random_state=42).reset_index(drop=True)
print('meta_df_scale shape:', meta_df_scale.shape)

# Train final LTR
from sklearn.model_selection import train_test_split
feat_cols = [c for c in meta_df_scale.columns if c not in ('user_idx','item_idx','label')]
X = meta_df_scale[feat_cols].values; y = meta_df_scale['label'].values
users_all = meta_df_scale['user_idx'].unique(); train_u, valid_u = train_test_split(list(users_all), test_size=0.10, random_state=42)
tr_mask = meta_df_scale['user_idx'].isin(train_u); val_mask = meta_df_scale['user_idx'].isin(valid_u)
X_tr = meta_df_scale[tr_mask][feat_cols].values; y_tr = meta_df_scale[tr_mask]['label'].values; gr_tr = meta_df_scale[tr_mask].groupby('user_idx', sort=False).size().astype(int).values
X_val = meta_df_scale[val_mask][feat_cols].values; y_val = meta_df_scale[val_mask]['label'].values; gr_val = meta_df_scale[val_mask].groupby('user_idx', sort=False).size().astype(int).values

dtrain = lgb.Dataset(X_tr, label=y_tr, group=gr_tr); dval = lgb.Dataset(X_val, label=y_val, group=gr_val, reference=dtrain)
params_scale = {'objective':'lambdarank','metric':'ndcg','ndcg_eval_at':[10,50],'learning_rate':float(best_params_proto.get('learning_rate',0.05)),'num_leaves':int(best_params_proto.get('num_leaves',63)),'min_data_in_leaf':int(best_params_proto.get('min_data_in_leaf',20)),'feature_fraction':float(best_params_proto.get('feature_fraction',0.8)),'verbosity':-1,'seed':42,'feature_pre_filter':False}
bst_final_scale = lgb.train(params_scale, dtrain, num_boost_round=1000, valid_sets=[dval], callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=100)])
joblib.dump(bst_final_scale, f"{OUT_DIR}/lgbm_reranker_final.joblib")

Building Meta Scale:   0%|          | 0/1561 [00:00<?, ?it/s]

meta_df_scale shape: (129516, 16)
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[46]	valid_0's ndcg@10: 0.949006	valid_0's ndcg@50: 0.949646


['/kaggle/working/lgbm_reranker_final.joblib']

## 7 — Final Evaluation & Metrics

Compute P@K, R@K, NDCG@K, MAP, and MPR for the LGBM hybrid reranker and a semantic baseline. Results are saved to `OUT_DIR/final_scale_summary_metrics.csv`.

### Explanation for Cell 19

**Purpose:** Imports libraries used later. Defines function(s) used downstream. Produces predictions on data. Saves results or models to disk.

**Line-by-line (short):**
- `# Final evaluation utilities and run` → Comment describing intent.
- `from math import log2` → Imports module(s).
- `def ndcg_at_k(pred_list, true_list, k=10):` → Performs a step relevant to data processing or modelling.
- `if not true_list: return 0.0` → Performs a step relevant to data processing or modelling.
- `pred_k = pred_list[:k]; gains = [1.0 if p in true_list else 0.0 for p in pred_k]` → Performs a step relevant to data processing or modelling.
- `dcg = sum(g / log2(i+2) for i,g in enumerate(gains)); idcg = sum(1.0 / log2(i+2) for i in range(min(len(true_list), k)))` → Performs a step relevant to data processing or modelling.
- `return dcg/idcg if idcg>0 else 0.0` → Performs a step relevant to data processing or modelling.
- `def apk(actual, predicted, k=10):` → Performs a step relevant to data processing or modelling.
- `if not actual: return 0.0` → Performs a step relevant to data processing or modelling.
- `score = 0.0; num_hits = 0.0` → Performs a step relevant to data processing or modelling.
- `for i,p in enumerate(predicted[:k]):` → Performs a step relevant to data processing or modelling.
- `if p in actual and p not in predicted[:i]:` → Performs a step relevant to data processing or modelling.

...remaining lines omitted for brevity. See code cell for full content.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [11]:
# Final evaluation utilities and run
from math import log2
def ndcg_at_k(pred_list, true_list, k=10):
    if not true_list: return 0.0
    pred_k = pred_list[:k]; gains = [1.0 if p in true_list else 0.0 for p in pred_k]
    dcg = sum(g / log2(i+2) for i,g in enumerate(gains)); idcg = sum(1.0 / log2(i+2) for i in range(min(len(true_list), k)))
    return dcg/idcg if idcg>0 else 0.0

def apk(actual, predicted, k=10):
    if not actual: return 0.0
    score = 0.0; num_hits = 0.0
    for i,p in enumerate(predicted[:k]):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0; score += num_hits/(i+1.0)
    return score/min(len(actual), k)

def lgbm_final_topk(uidx, k=10):
    cands = build_candidates_union(uidx, item_emb_final, (als_model_final if 'als_model_final' in globals() else None), svd_item_factors_final, item_map_scale, pop_counts_scale, user_map_scale, df_scale, interaction_scale, top_sem=1000, top_als=500, top_svd=500, top_pop=500, top_mf=200, max_cands=3000)
    if not cands: return []
    mf = build_features_for_candidates(uidx, cands, item_emb_final, (als_item_factors_final if 'als_item_factors_final' in globals() else None), (als_user_factors_final if 'als_user_factors_final' in globals() else None), svd_item_factors_final, df_scale, item_map_scale, user_map_scale, pop_counts_scale, feature_names, now_ts)
    preds = bst_final_scale.predict(mf['X']); order = np.argsort(-preds)[:k]; return [cands[i] for i in order]

def semantic_final_topk(uidx, k=10):
    uvec = user_profile_emb(uidx, item_emb_final, df_scale, item_map_scale, user_map_scale); uvec = uvec/(np.linalg.norm(uvec)+1e-9)
    scores = item_emb_final.dot(uvec); return list(np.argsort(-scores)[:k])

def evaluate_final(scorer_fn, K=10, max_users=1000):
    actuals=[]; preds=[]; cnt=0
    for uid in tqdm(test_df_scale['reviewerID'].unique()[:max_users], desc=f"Evaluating {scorer_fn.__name__}"):
        if uid not in user_map_scale: continue
        uidx = user_map_scale[uid]
        true_asins = test_df_scale[test_df_scale['reviewerID']==uid]['asin'].tolist()
        if not true_asins: continue
        rec_idxs = scorer_fn(uidx, k=K)
        rec_asins = [inv_item_map_scale[i] for i in rec_idxs if i in inv_item_map_scale]
        preds.append(rec_asins); actuals.append(true_asins); cnt += 1
    if cnt==0: return {'P@10':0.0,'R@10':0.0,'NDCG@10':0.0,'MAP@10':0.0,'MPR@10':0.0,'EvalUsers':0}
    P = np.mean([len(set(p[:K]) & set(a))/float(K) for p,a in zip(preds,actuals)])
    R = np.mean([len(set(p[:K]) & set(a))/max(1,len(a)) for p,a in zip(preds,actuals)])
    N = np.mean([ndcg_at_k(p,a,K) for p,a in zip(preds,actuals)])
    MAP = np.mean([apk(a,p,K) for a,p in zip(actuals,preds)])
    MPR = np.mean([np.mean([1.0 - (p.index(t)+1)/K if (t in p[:K]) else 0.0 for t in a]) for a,p in zip(actuals,preds)])
    return {'P@10':P,'R@10':R,'NDCG@10':N,'MAP@10':MAP,'MPR@10':MPR,'EvalUsers':len(actuals)}

final_results = {}
final_results['LGBM_Hybrid_Rerank'] = evaluate_final(lgbm_final_topk, K=10)
final_results['Semantic_Baseline'] = evaluate_final(semantic_final_topk, K=10)
final_df = pd.DataFrame(final_results).T
final_df = final_df[['P@10','R@10','NDCG@10','MAP@10','MPR@10','EvalUsers']]
final_df.to_csv(f"{OUT_DIR}/final_scale_summary_metrics.csv", index=False)
print('Final evaluation complete — metrics saved to', f"{OUT_DIR}/final_scale_summary_metrics.csv")

Evaluating lgbm_final_topk:   0%|          | 0/1000 [00:00<?, ?it/s]

Evaluating semantic_final_topk:   0%|          | 0/1000 [00:00<?, ?it/s]

Final evaluation complete — metrics saved to /kaggle/working/final_scale_summary_metrics.csv


### Feature importance

Export and inspect feature importances from the final LightGBM model.

### Explanation for Cell 21

**Purpose:** Saves results or models to disk.

**Line-by-line (short):**
- `# Feature importances` → Comment describing intent.
- `fi_df = pd.DataFrame({'feature': feat_cols, 'importance': bst_final_scale.feature_importance()}).sort_values('importance', ascending=False)` → Performs a step relevant to data processing or modelling.
- `fi_df.to_csv(f"{OUT_DIR}/feature_importances_final.csv", index=False)` → Performs a step relevant to data processing or modelling.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [12]:
# Feature importances
fi_df = pd.DataFrame({'feature': feat_cols, 'importance': bst_final_scale.feature_importance()}).sort_values('importance', ascending=False)
fi_df.to_csv(f"{OUT_DIR}/feature_importances_final.csv", index=False)

### Explanation for Cell 22

**Purpose:** Performs an operation relevant to the pipeline (see code).

**Line-by-line (short):**
- `# Fixed sample recommendation printing (maps indices -> ASIN strings, computes true HITs)` → Comment describing intent.
- `SAMPLE_USERS_TO_DISPLAY = 5` → Performs a step relevant to data processing or modelling.
- `test_users = test_df_scale['reviewerID'].unique()` → Performs a step relevant to data processing or modelling.
- `if len(test_users) == 0:` → Performs a step relevant to data processing or modelling.
- `print("\nNo unique test users found to display sample recommendations.")` → Displays a quick diagnostic for the user.
- `else:` → Performs a step relevant to data processing or modelling.
- `num_samples = min(SAMPLE_USERS_TO_DISPLAY, len(test_users))` → Performs a step relevant to data processing or modelling.
- `sample_uids = random.sample(list(test_users), num_samples)` → Samples rows from a DataFrame.
- `print(f"\n--- Sample User Recommendation Output (LGBM Hybrid) for {num_samples} Users ---")` → Displays a quick diagnostic for the user.
- `for sample_uid in sample_uids:` → Performs a step relevant to data processing or modelling.
- `sample_uidx = user_map_scale.get(sample_uid)` → Performs a step relevant to data processing or modelling.
- `if sample_uidx is None:` → Performs a step relevant to data processing or modelling.

...remaining lines omitted for brevity. See code cell for full content.

**Inputs:** likely dataframes, parameters, and previously defined variables.
**Outputs:** new variables in memory (dataframes, models, figures), possibly files on disk.

**Notes:** Verify paths, column names, and variable names. Consider adding assertions to safeguard assumptions.


In [13]:

# Fixed sample recommendation printing (maps indices -> ASIN strings, computes true HITs)
SAMPLE_USERS_TO_DISPLAY = 5

test_users = test_df_scale['reviewerID'].unique()
if len(test_users) == 0:
    print("\nNo unique test users found to display sample recommendations.")
else:
    num_samples = min(SAMPLE_USERS_TO_DISPLAY, len(test_users))
    sample_uids = random.sample(list(test_users), num_samples)

    print(f"\n--- Sample User Recommendation Output (LGBM Hybrid) for {num_samples} Users ---")

    for sample_uid in sample_uids:
        sample_uidx = user_map_scale.get(sample_uid)

        if sample_uidx is None:
            print(f"\n[SKIP] User {sample_uid} not in user_map_scale.")
            continue

        if 'bst_final_scale' not in globals() or bst_final_scale is None:
            print("\n[SKIP] bst_final_scale not available — model wasn't trained or loaded.")
            break

        # true ASIN string
        actual_asin = test_df_scale[test_df_scale['reviewerID'] == sample_uid]['asin'].iloc[0]

        # get recommended item indices (ints)
        rec_idxs = lgbm_final_topk(sample_uidx, k=10)

        # convert indices -> ASINs (use appropriate inv map)
        inv_map = inv_item_map_scale if 'inv_item_map_scale' in globals() else inv_item_map
        rec_asins = [inv_map[i] if (i in inv_map) else f"IDX_{i}" for i in rec_idxs]

        print(f"\nUser ID: {sample_uid}")
        print(f"True Last Item (to predict): {actual_asin}")
        print("Top 10 Recommended ASINs:")
        for i, asin in enumerate(rec_asins):
            hit_status = "✅ HIT" if asin == actual_asin else "❌ MISS"
            print(f"  {i+1}. {asin} ({hit_status})")

        # helpful additional info: was the true item present in the candidate set at all?
        # Build candidate set with the same function used by the scorer (using larger pool for clarity)
        cands = build_candidates_union(sample_uidx,
                                       item_emb_final if 'item_emb_final' in globals() else None,
                                       als_model_final if 'als_model_final' in globals() else None,
                                       svd_item_factors_final if 'svd_item_factors_final' in globals() else None,
                                       item_map_scale if 'item_map_scale' in globals() else item_map,
                                       pop_counts_scale if 'pop_counts_scale' in globals() else pop_counts,
                                       user_map_scale if 'user_map_scale' in globals() else user_map,
                                       df_scale if 'df_scale' in globals() else df,
                                       interaction_scale if 'interaction_scale' in globals() else interaction,
                                       top_sem=1000, top_als=500, top_svd=500, top_pop=500, top_mf=200, max_cands=3000)
        cand_asins = {inv_map[i] for i in cands if i in inv_map}
        in_candidates = actual_asin in cand_asins
        print(f"True ASIN in candidate pool? {'YES' if in_candidates else 'NO'}")



--- Sample User Recommendation Output (LGBM Hybrid) for 5 Users ---

User ID: AFANVB6MPHJTCTFOVIEBKLWZ2GVA
True Last Item (to predict): B07MCMTCP3
Top 10 Recommended ASINs:
  1. B07MCSKB9P (❌ MISS)
  2. B00KJLMBQQ (❌ MISS)
  3. B096SV8SJG (❌ MISS)
  4. B07J2Z5DBM (❌ MISS)
  5. B071WGSZ81 (❌ MISS)
  6. B07RGZ5NKS (❌ MISS)
  7. B07ZPC9QD4 (❌ MISS)
  8. B008X0TBKO (❌ MISS)
  9. B0001FTVEK (❌ MISS)
  10. B08H1WNYTR (❌ MISS)
True ASIN in candidate pool? NO

User ID: AH6PLOGWYIVIWLJTY756BHNFD4YA
True Last Item (to predict): B0068LWHPQ
Top 10 Recommended ASINs:
  1. B0036ZZ72W (❌ MISS)
  2. B0018B5CA8 (❌ MISS)
  3. B07M66HC8K (❌ MISS)
  4. B076CRF7DN (❌ MISS)
  5. B075WD3YHX (❌ MISS)
  6. B00KIVGGCC (❌ MISS)
  7. B07CXG6C9W (❌ MISS)
  8. B01MEG2HXP (❌ MISS)
  9. B07H9HGTC6 (❌ MISS)
  10. B07MLV7TRJ (❌ MISS)
True ASIN in candidate pool? YES

User ID: AHTRY6YXVNYOGRTUYANAI2M7DUEA
True Last Item (to predict): B00NH11H38
Top 10 Recommended ASINs:
  1. B013TMNNDQ (❌ MISS)
  2. B07XCNK22K (❌ MISS)

## 8 — Diagnostics, Ablations & Next Steps

**Diagnostics included in this notebook:**

- Candidate recall checks (small and large pools)
- Feature distributions for positives vs negatives
- Per-user inspection utilities (`inspect_user_rerank`, `compare_top_bottom`)

**Suggested next steps:**

1. If reranker underperforms but recall is high: increase meta training size, upsample positives, and tune hard-negative ratios.
2. If ALS/SVD features are weak, consider training implicit ALS on denser data or using matrix factorization with alternating optimization.
3. Add more content features (title tokens, brand, category) and context signals (time of day, device) if available.
4. For production: build offline candidate pipeline (FAISS HNSW) and a lightweight online reranker (optimized model export).

---



In [14]:
# Cleanly load & display the saved final metrics table, then give number-free explanations
import os
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

# Candidate CSV paths to try (prefer OUT_DIR if available)
csv_candidates = []
if 'OUT_DIR' in globals() and OUT_DIR:
    csv_candidates.append(os.path.join(OUT_DIR, "final_scale_summary_metrics.csv"))
csv_candidates.extend([
    "final_scale_summary_metrics.csv",
    os.path.join(os.getcwd(), "final_scale_summary_metrics.csv"),
])

# Find a readable CSV
metrics_df = None
csv_used = None
for p in csv_candidates:
    if p and os.path.exists(p):
        try:
            df_try = pd.read_csv(p)
            # Basic heuristics for readability: prefer files with expected metric columns
            metrics_df = df_try.copy()
            csv_used = p
            break
        except Exception as e:
            display(Markdown(f"**Found file but failed to read** `{p}` — error: `{e}`"))

# If not found, show friendly error
if metrics_df is None:
    display(Markdown("❌ **Could not find or read** `final_scale_summary_metrics.csv` in OUT_DIR or current directory. "
                     "Make sure the notebook saved it earlier."))
else:
    # ---- CLEANUP ----
    df = metrics_df.copy()

    # 1) Drop obvious accidental index column (Unnamed: 0 or first column named '0')
    if df.columns[0].startswith("Unnamed") or df.columns[0] == "0":
        df = df.iloc[:, 1:].copy()

    # 2) If columns are metrics but are rows (transposed), try to detect and transpose
    expected_metrics = {'P@10','R@10','NDCG@10','MAP@10','MPR@10','EvalUsers'}
    cols_set = set(df.columns.astype(str))
    # If the expected metrics are not in columns but are in the first column values, transpose.
    if not expected_metrics.intersection(cols_set):
        # check if first column contains metric names
        first_col_vals = set(df.iloc[:,0].astype(str).str.strip().tolist())
        if expected_metrics.intersection(first_col_vals):
            try:
                df = df.set_index(df.columns[0]).T.reset_index(drop=True)
            except Exception:
                pass

    # 3) Ensure numeric columns are numeric
    for c in df.columns:
        # try to coerce numeric-like columns
        try:
            df[c] = pd.to_numeric(df[c], errors='ignore')
        except Exception:
            pass

    # 4) If no friendly model names in index, create readable labels
    if df.index.dtype != 'object' or all(isinstance(i, (int, np.integer, float)) for i in df.index):
        # If a 'Model' column exists, use it; otherwise create Model_1, Model_2...
        if 'Model' in df.columns:
            df = df.set_index('Model')
        else:
            df.index = [f"Model_{i+1}" for i in range(len(df))]

    # 5) Reorder columns to expected order where possible
    desired_order = ['P@10','R@10','NDCG@10','MAP@10','MPR@10','EvalUsers']
    present = [c for c in desired_order if c in df.columns]
    others = [c for c in df.columns if c not in present]
    df = df[present + others]

    # ---- DISPLAY CLEAN TABLE ----
    display(Markdown(f"## 📊 Final evaluation metrics, feeding 1 Million rows to both models and comparing (loaded from `{csv_used}`)"))
    # Format numeric display: 6 decimals for floats, integer for EvalUsers if present
    fmt = {}
    for c in df.columns:
        if c == 'EvalUsers':
            fmt[c] = "{:.0f}"
        else:
            # only apply float format if column is numeric
            if pd.api.types.is_numeric_dtype(df[c]):
                fmt[c] = "{:.6f}"
            else:
                fmt[c] = "{}"
    display(df.style.format(fmt).set_caption("Cleaned final scale summary metrics"))

    # ---- QUALITATIVE COMPARISONS (NO RAW NUMBERS) ----
    present_metric_cols = [c for c in ['P@10','R@10','NDCG@10','MAP@10','MPR@10'] if c in df.columns]
    qualitative_notes = []

    if len(df) == 1:
        qualitative_notes.append("Only a single model/row is present — no cross-model comparison available.")
    else:
        for col in present_metric_cols:
            try:
                # Determine best model(s) for this metric (higher is better)
                best_val = df[col].max()
                best_models = df.index[df[col] == best_val].tolist()
                if len(best_models) == 1:
                    qualitative_notes.append(f"- **{col}**: top performer — **{best_models[0]}**.")
                else:
                    qualitative_notes.append(f"- **{col}**: top performers (tie) — **{', '.join(best_models)}**.")
            except Exception:
                # skip if comparison fails for a column
                pass

    # ---- HUMAN-FRIENDLY EXPLANATION (NO HARD THRESHOLDS) ----
    explanation_lines = [
        "### 📘 Metric meanings (guidance — no numeric thresholds)",
        "",
        "- **Precision@10 (P@10)** — Of the top-10 items returned, the fraction that are relevant. In very large catalogs, absolute Precision values can be low while still being useful.",
        "",
        "- **Recall@10 (R@10)** — Fraction of the user's relevant items that appear in the top-10. High Recall means good coverage of what the user is likely to want.",
        "",
        "- **NDCG@10** — Rewards placing the most relevant items near the top. This is often the primary ranking metric to optimize when user satisfaction depends on top-list order.",
        "",
        "- **MAP@10** — Mean Average Precision across users; emphasizes consistent early correct placements.",
        "",
        "- **MPR@10** — A rank-based measure capturing how near the top relevant items are on average; complements NDCG/MAP.",
        "",
        "- **EvalUsers** — The number of users used for evaluation; larger is more reliable.",
        "",
        "### 🔎 How to interpret combinations (qualitative)",
        "",
        "- If **NDCG** and **MAP** improve together, the model provides reliably better top-ranked results across users.",
        "- If a model shows higher **Recall** but lower **Precision**, it finds many relevant items but the top list may include more noise — consider stronger reranking or feature signals.",
        "- A modest Precision in a huge catalog can still be valuable if **Recall** and **NDCG** are strong — the system surfaces relevant items and ranks them well even if many slots remain non-relevant.",
        "",
        "### ✅ Practical next steps (if you want to improve results)",
        "",
        "- Verify candidate recall (if many true items never enter the candidate pool, increase pool size or add ANN).",
        "- Improve hard-negative mining diversity to teach the reranker harder distinctions.",
        "- Add more features (metadata, temporal signals) to the LTR training data.",
        "- Consider using stronger encoders (on GPU) or FAISS for ANN retrieval if compute allows."
    ]

    # ---- OUTPUT ----
    if qualitative_notes:
        display(Markdown("### 🔍 Qualitative model winners (per metric)"))
        display(Markdown("\n".join(qualitative_notes)))
    display(Markdown("\n".join(explanation_lines)))



/tmp/ipykernel_13/3532202218.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors='ignore')


## 📊 Final evaluation metrics, feeding 1 Million rows to both models and comparing (loaded from `/kaggle/working/final_scale_summary_metrics.csv`)

,P@10,R@10,NDCG@10,MAP@10,MPR@10,EvalUsers
Model_1,0.052800,0.528000,0.452686,0.429620,0.422300,1000
Model_2,0.073000,0.730000,0.624721,0.589946,0.605900,1000


### 🔍 Qualitative model winners (per metric)

- **P@10**: top performer — **Model_2**.
- **R@10**: top performer — **Model_2**.
- **NDCG@10**: top performer — **Model_2**.
- **MAP@10**: top performer — **Model_2**.
- **MPR@10**: top performer — **Model_2**.

### 📘 Metric meanings (guidance — no numeric thresholds)

- **Precision@10 (P@10)** — Of the top-10 items returned, the fraction that are relevant. In very large catalogs, absolute Precision values can be low while still being useful.

- **Recall@10 (R@10)** — Fraction of the user's relevant items that appear in the top-10. High Recall means good coverage of what the user is likely to want.

- **NDCG@10** — Rewards placing the most relevant items near the top. This is often the primary ranking metric to optimize when user satisfaction depends on top-list order.

- **MAP@10** — Mean Average Precision across users; emphasizes consistent early correct placements.

- **MPR@10** — A rank-based measure capturing how near the top relevant items are on average; complements NDCG/MAP.

- **EvalUsers** — The number of users used for evaluation; larger is more reliable.

### 🔎 How to interpret combinations (qualitative)

- If **NDCG** and **MAP** improve together, the model provides reliably better top-ranked results across users.
- If a model shows higher **Recall** but lower **Precision**, it finds many relevant items but the top list may include more noise — consider stronger reranking or feature signals.
- A modest Precision in a huge catalog can still be valuable if **Recall** and **NDCG** are strong — the system surfaces relevant items and ranks them well even if many slots remain non-relevant.

### ✅ Practical next steps (if you want to improve results)

- Verify candidate recall (if many true items never enter the candidate pool, increase pool size or add ANN).
- Improve hard-negative mining diversity to teach the reranker harder distinctions.
- Add more features (metadata, temporal signals) to the LTR training data.
- Consider using stronger encoders (on GPU) or FAISS for ANN retrieval if compute allows.